In [1]:
import torch

In [2]:
import torch
print(torch.cuda.is_available())


True


In [3]:
import pandas as pd

In [37]:
train_dataset=pd.read_csv("/content/fashion-mnist_train.csv")

In [38]:
train_dataset.shape

(60000, 785)

In [39]:
train_features=train_dataset.drop(columns=["label"]).values
train_labels=train_dataset["label"].values

In [40]:
train_features=train_features/255.0

In [41]:
test_dataset=pd.read_csv("/content/fashion-mnist_test.csv")

In [42]:
test_dataset.shape

(10000, 785)

In [43]:
test_features=test_dataset.drop(columns=["label"]).values
test_labels=test_dataset["label"].values

In [44]:
test_features=test_features/255.0

In [45]:
test_dataset.shape

(10000, 785)

In [46]:
from torch.utils.data import Dataset,DataLoader

In [47]:
class customdataset(Dataset):
  def __init__(self,features,labels):
    self.features=torch.tensor(features,dtype=torch.float32)
    self.labels=torch.tensor(labels,dtype=torch.long)
  def __len__(self):
    return len(self.features)
  def __getitem__(self,index):
    return self.features[index],self.labels[index]


In [48]:
train_datasetss=customdataset(train_features,train_labels)
test_datasetss=customdataset(test_features,test_labels)

In [49]:
import numpy as np
np.unique(train_labels)





array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [50]:
train_labels.shape

(60000,)

In [51]:
len(train_datasetss)

60000

In [52]:
len(test_datasetss)

10000

In [53]:
train_datasetss[0]

(tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0157, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.2431, 0.2392, 0.0824, 0.1137, 0.0902,
         0.2000, 0.5333, 0.2392, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0

In [54]:
train_dataloader=DataLoader(train_datasetss,batch_size=32,shuffle=True,pin_memory=True)
test_dataloader=DataLoader(test_datasetss,batch_size=32,shuffle=False,pin_memory=True)


In [55]:
len(train_dataloader)## it return no of batch formed

1875

In [56]:
import torch.nn as nn

In [58]:
## create model
class models(nn.Module):
  def __init__(self,num_features):
    super().__init__()
    self.network=nn.Sequential(
        nn.Linear(num_features,128),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Dropout(p=0.3),
        nn.Linear(128,64),
        nn.BatchNorm1d(64),
        nn.ReLU(),
        nn.Dropout(p=0.3),
        nn.Linear(64,10)
    )
  def forward(self,features):
    out=self.network(features)
    return out

In [59]:
epochs=100
learning_rate=0.0001

In [60]:
model=models(train_features.shape[1])


In [61]:
## move to gpu first step is
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [62]:
model=model.to(device)

In [64]:
## intialize loss function
criterion=nn.CrossEntropyLoss()

In [65]:
## OPtmizers
import torch.optim as optim

optimizer = torch.optim.Adam(model.parameters(), lr=0.001,weight_decay=1e-4)


In [66]:
for batch_features, batch_labels in train_dataloader:
    print("Batch features:", batch_features.shape)
    print("Batch labels:", batch_labels.shape, batch_labels.dtype)
    print("Unique labels in this batch:", torch.unique(batch_labels))
    break


Batch features: torch.Size([32, 784])
Batch labels: torch.Size([32]) torch.int64
Unique labels in this batch: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


In [67]:
for epoch in range(epochs):
  total_epoch_loss=0
  for batch_features,batch_labels in train_dataloader:
    batch_features=batch_features.to(device)
    batch_labels=batch_labels.to(device)
    output=model(batch_features)
    loss=criterion(output,batch_labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    total_epoch_loss+=loss.item()
  avg_epoch_loss=(total_epoch_loss/len(train_dataloader))
  print(f'epoch:{epoch+1},loss:{avg_epoch_loss}')

epoch:1,loss:0.604262459675471
epoch:2,loss:0.4676065255244573
epoch:3,loss:0.4309376077890396
epoch:4,loss:0.41380290442705153
epoch:5,loss:0.40176212318340937
epoch:6,loss:0.392348731815815
epoch:7,loss:0.3860754104534785
epoch:8,loss:0.37729070026874545
epoch:9,loss:0.36990643508434295
epoch:10,loss:0.36941631443500517
epoch:11,loss:0.3640300684769948
epoch:12,loss:0.35771856621901194
epoch:13,loss:0.3572214227716128
epoch:14,loss:0.35477863289515177
epoch:15,loss:0.35201876100301743
epoch:16,loss:0.35166578789949415
epoch:17,loss:0.34677111669778826
epoch:18,loss:0.3454489293575287
epoch:19,loss:0.3491455854614576
epoch:20,loss:0.34049657417933143
epoch:21,loss:0.3407856152375539
epoch:22,loss:0.3392789019306501
epoch:23,loss:0.33737975028355915
epoch:24,loss:0.33708245458602903
epoch:25,loss:0.33645411644379297
epoch:26,loss:0.3378204295873642
epoch:27,loss:0.3328362860659758
epoch:28,loss:0.332101698700587
epoch:29,loss:0.3312212957918644
epoch:30,loss:0.33155102993647256
epoch:3

In [32]:
## model evaluation

In [68]:
model.eval()

models(
  (network): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.3, inplace=False)
    (8): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [69]:
total=0
correct=0
with torch.no_grad():

  for batch_features,batch_labels in test_dataloader:

    batch_features=batch_features.to(device)
    batch_labels=batch_labels.to(device)
    output=model(batch_features)
    _, predicted=torch.max(output,1)

    total+=batch_labels.shape[0]
    correct+=(predicted==batch_labels).sum().item()
  print(correct/total)

0.8947


In [35]:
## ened

In [70]:
for batch_features,batch_labels in train_dataloader:
  batch_features=batch_features.to(device)
  batch_labels=batch_labels.to(device)
  output=model(batch_features)
  _, predicted=torch.max(output,1)
  total+=batch_labels.shape[0]
  correct+=(predicted==batch_labels).sum().item()
print(correct/total)


0.9193857142857143


In [ ]:
## now overfitting reduce